Check Bash Version
------------------------------

In [ ]:
echo $BASH_VERSION

Variables
--------------

In [ ]:
myname="Hans Schmid"
echo "Hello, $myname"

In [ ]:
echo $myname

String interpolation

In [ ]:
echo "$myname"

Literal string

In [ ]:
echo '$MYNAME'

In [ ]:
type pwd

In [ ]:
echo $(pwd)

In [ ]:
echo $PWD

Backticks are regarded as obsolete. But hard to kill in the field.

In [ ]:
echo `pwd`

Keywords
---------------
Braces, double brackets and exclamation mark are keywords. That means in order to use them correctly you need spaces around them!

In [ ]:
type if then elif else fi time for in until while do done case esac coproc select function { } [[ ]] !

Equal sign is not a keyword. So when using it you do not put spaces around it.

In [ ]:
type =

which is obsolete. You'll find it a lot though.
<br>
**'which' is a bitch.**

In [ ]:
type which

Use **type -p** instead because it's a builtin!

In [ ]:
type -p which

In [ ]:
type type

# Functions

1. Functions in Bash are not functions like in other languages. They are actually commands. Functions are used as if they were command line binaries or scripts.
2. Shell commands are connected by pipes (aka streams), and not by fundamental or user-defined data types as in _real_ programming languages. There is no such thing as a return value for a command. (Although there is a return statement - see below.)
3. When a function wants to get input it reads it from its input stream, or the argument list. In both cases text strings have to be parsed. The shell with its pipe-based architecture is purely string-based!
4. When a command wants to return something it has to _echo/printf_ it to its output stream.
5. The caller can use command substitution _$()_ to capture the output.
6. Another practiced way is to store the return values in dedicated, global variables. (Writing to the output stream is clearer and more flexible, because it can also take binary data.)
7. Think of the exit code as a bonus that other languages don't have, or as a "Schmutzeffekt" of shell functions. The meaning of the exit code is up to the shell programmer!
8. Often by convention an exit status of 0 means success, 1-255 means failure and the exit code can be regarded as a failure number.
9. Just to make sure: _return/exit_ can only take a value from 0-255. Values other than 0 are _not necessarily_ errors. There is always an exit status.
10. The argument list is only for decoration and you never put anything inside them.

## How to define functions
There are two ways to define functions.

### 1. With the **_function_** keyword:
a) The argument list is optional.<br>
b) Coding style: Do not use an argument list.

In [ ]:
function success {
    echo "I am successful!"
    return 0
}

### 2. Without the **_function_** keyword.
The argument list is mandatory.

In [ ]:
success() {
    echo "I am successful!"
    return 0
}

### Passing and Accessing Arguments

In [ ]:
function printme {
    echo "You gave me $# argument(s)!"
    echo -n $1
}

In [ ]:
printme

In [ ]:
printme 'Hello'

In [ ]:
printme 'Hello' 'World'

### Returning Values

#### 1. echo/printf values to the output stream

In [ ]:
function add {
    # $((...)) to calculate mathematical expressions
    sum=$(($1+$2))
    echo -n $sum
}

In [ ]:
add 1 2

#### 1. Capture Return Values via Command Substition Using $()

In [ ]:
result=$(add 1 2)
echo $result

#### 2. Use Global Variables to Provide Return Values
The _sum_ variable in our add function is actually a global variable.

In [ ]:
echo -n $sum

### Variable Scope
Using local variables in order to not pollute the global namespace.

In [ ]:
function add_local {
    local sum=$(($1+$2))
    echo -n $sum
}

In [ ]:
add_local 5 6

The global variable from above did not change.

In [ ]:
echo -n $sum

#### 3. Argument References
As of Bash 4.3+, we can pass an input argument by reference.

In [ ]:
function add_by_ref {
    declare -n add_ref=$3
    add_ref=$(($1+$2)) 
}

In [ ]:
add_by_ref 1 2 myAddRefResult

In [ ]:
echo -n $myAddRefResult

### Sub-shells
1. A sub-shell is a special type of command group that allows us to spawn a new execution environment from the current shell.
2. Instead of curly braces, we use parentheses to delimit the function body.

In [ ]:
function add_subshell (
    # 'sum' is a global variable
    sum=$(($1+$2))
    echo -n $sum 
)

In [ ]:
add_subshell 2 3

Our global variable did not change because the function was executed in a sub-shell.

In [ ]:
echo -n $sum

### Recursion
Recursion is possible but not typical for shell programming.

In [ ]:
function factorial {
    if [ $1 -le 1 ]; then
        echo -n 1
    else
        echo -n $(($(factorial $(($1-1)))*$1))
    fi 
}

In [ ]:
factorial 5

In [ ]:
factorial 10

# Arrays

## Declaring Arrays
There are two ways:<br>
1. Declare a variable explicitly to be an array.
2. Create an array on the fly.

#### Declaring an array explicitly

In [ ]:
declare -p pioneers

In [ ]:
declare -a pioneers

In [ ]:
declare -p pioneers

#### Creating an Array on the Fly
An array is automatically created when you assign a value to a variable.

In [ ]:
pioneers[0]='Ken Thompson'

In [ ]:
declare -p pioneers

You can assign multiple values at once.

In [ ]:
unset pioneers

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie')

In [ ]:
declare -p pioneers

#### Assigning Values to an Array
1. An array doesn't need to have continuous indexes.
2. When you try to print an array element that is not initialized, you'll get a null value.

In [ ]:
pioneers[20]='Douglas McIlroy'

In [ ]:
declare -p pioneers

### Accessing Array Elements

The default index is zero.

In [ ]:
echo $pioneers

This does not work as one might expect coming from a mainstream programming language.

In [ ]:
echo $pioneers[1]

This is the correct way.

In [ ]:
echo ${pioneers[1]}

Now the whole array:

In [ ]:
echo ${pioneers[@]}

In [ ]:
echo ${pioneers[*]}

We"ll see the difference of the two commands above when we cover looping.

In [ ]:
declare -p pioneers

### Appending an Array Element

In [ ]:
unset pioneers

You can also explicitly specify the index of the array entry.

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]='Douglas McIlroy')

In [ ]:
pioneers+=('Linus Torvals' 'Richard Stallman')

In [ ]:
declare -p pioneers

You often see this workaround in older scripts:

In [ ]:
unset pioneers

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

In [ ]:
declare -p pioneers

This workaround of appending array elements works only when indices are consecutive.

In [ ]:
pioneers=("${pioneers[@]}" 'Joe Ossanna' 'Linus Torvalds')

In [ ]:
declare -p pioneers

The quotes are essential!

In [ ]:
pioneers=(${pioneers[@]} 'Richard Stallman')

In [ ]:
declare -p pioneers

### Slicing Arrays

In [ ]:
unset pioneers

Without using quotes.

In [ ]:
pioneers=(Ken Brian Dennis Douglas)

First parameter is index, second parameter is length!
Note the parens!

In [ ]:
pioneers2=(${pioneers[@]:1:2})

In [ ]:
declare -p pioneers2

Not what you expected:

In [ ]:
pioneers2=${pioneers[@]:1:2}

In [ ]:
declare -p pioneers2

Slice from the beginning:

In [ ]:
pioneers2=(${pioneers[@]::2})

In [ ]:
declare -p pioneers2

Slice from the end:

In [ ]:
pioneers2=(${pioneers[@]:2})

In [ ]:
declare -p pioneers2

### Getting Indices

In [ ]:
pioneers=(Ken Brian Dennis Douglas)

In [ ]:
echo -n ${!pioneers[@]}

In [ ]:
pioneers=(Ken Brian Dennis Douglas [10]=Joe)

In [ ]:
echo -n ${!pioneers[@]}

### Removing Array Elements

In [36]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

: 1

Bye, bye, Brian!

In [ ]:
unset pioneers[1]

In [ ]:
declare -p pioneers

### Array Length

In [ ]:
echo -n ${#pioneers[@]}

In [ ]:
echo -n ${#pioneers[*]}

In [ ]:
echo -n 'Length of the third element:' ${#pioneers[2]}

### Looping through an Array

In [ ]:
i=1
for item in "${pioneers[@]}"; do
    echo $((i++)). $item
done

In [34]:
i=1
for item in "${pioneers[*]}"; do
    echo $((i++)). $item
done

1. Ken Thompson Brian Kernighan Dennis Ritchie Douglas McIlroy


: 1

In [45]:
for index in ${!pioneers[@]}; do
    echo -e 'Index: '$index'\t'${pioneers[$index]}
done

Index: 0	Ken Thompson04l
Index: 1	Brian Kernighan
Index: 2	Dennis Ritchie
Index: 20	Douglas McIlroy


: 1

#### Using File Globbing to Initialize an Array

In [ ]:
files=(/etc/[abcdefg]*.conf)

In [ ]:
declare -p files

In [ ]:
i=1
for file in ${files[@]}; do
    echo $((i++)). $file
done

#### Loading File Content into an Array

In [ ]:
unset pioneers

In [ ]:
cat pioneers.txt

In [ ]:
pioneers=( $(cat pioneers.txt) )

In [ ]:
declare -p pioneers

In [ ]:
for pioneer in ${pioneers[@]}; do
    echo $pioneer
done

#### Loading Arrays from a File with _mapfile_

In [28]:
mapfile -t pioneers < pioneers.txt # -t strips newline

: 1

In [29]:
declare -p pioneers

declare -a pioneers=([0]="Ken" [1]="Dennis" [2]="Brian" [3]="Douglas" [4]="Joe")


: 1

You can leave out an array name. Bash will use MAPFILE in this case.

In [30]:
mapfile -t < pioneers.txt

: 1

In [31]:
declare -p MAPFILE

declare -a MAPFILE=([0]="Ken" [1]="Dennis" [2]="Brian" [3]="Douglas" [4]="Joe")


: 1

# Associate Arrays

## Differences from Arrays
1. declare -A (seems to be mandatory!)
2. Index is a string, not a number (0,1,2,3,..)

In [3]:
declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [C++]="Bjarne Stroustrup")

: 1

In [4]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [C++]="Bjarne Stroustrup" )


: 1

In [5]:
echo -n ${!proglangs[@]}

Python Ruby C++

: 1

In [6]:
echo ${proglangs[Python]}

Guido van Rossum


: 1

In [8]:
proglangs[Erlang]="Joe Armstrong"

: 1

In [9]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [Erlang]="Joe Armstrong" [C++]="Bjarne Stroustrup" )


: 1

In [10]:
proglangs+=([Javascript]="Brendan Eich" [Perl]="Larry Wall")

: 1

In [11]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [Perl]="Larry Wall" [Erlang]="Joe Armstrong" [Javascript]="Brendan Eich" [C++]="Bjarne Stroustrup" )


: 1

In [12]:
echo -n ${!proglangs[@]}

Python Ruby Perl Erlang Javascript C++

: 1

In [13]:
echo -n ${#proglangs[@]}

6

: 1

In [14]:
unset proglangs[Javascript]

: 1

In [15]:
echo -n ${#proglangs[@]}

5

: 1

In [16]:
echo -n ${!proglangs[@]}

Python Ruby Perl Erlang C++

: 1

In [18]:
i=1
for proglang in "${proglangs[@]}"; do
    echo $((i++)). $proglang
done

1. Guido van Rossum2004l
2. Yukihiro Matsumoto
3. Larry Wall
4. Joe Armstrong
5. Bjarne Stroustrup


: 1

In [31]:
for key in ${!proglangs[@]}; do
    echo -e $key'\t'${proglangs[$i]}
done

Python	Bjarne Stroustrup
Ruby	Bjarne Stroustrup
Perl	Bjarne Stroustrup
Erlang	Bjarne Stroustrup
C++	Bjarne Stroustrup


: 1

# Loops
Looping constructs in Bash:
1. For Loops
2. For Loops/C-Syntax
3. While Loops
4. Until Loops
5. Case Command
6. Select Command

## For Loops
General structure:
<br><code>
for varname in list
do
    commands using $varname"
done
</code>

In [37]:
for day in Mon Tue Wed Thu Fri; do
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed
Weekday: Thu
Weekday: Fri


: 1

In [38]:
weekdays="Mon Tue Wed Thu Fri"
for day in $weekdays; do
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed
Weekday: Thu
Weekday: Fri


: 1

But not:

In [39]:
weekdays="Mon Tue Wed Thu Fri"
for day in "$weekdays"; do
    echo "Weekday: $day"
done

Weekday: Mon Tue Wed Thu Fri004l


: 1

In [41]:
for username in $(awk -F: '{print $1}' /etc/passwd); do
    echo "Username: $username"
done

Username: root4l
Username: nobody
Username: _uuidd
Username: schmidh
Username: _dhcpcd
Username: polkitd
Username: dbus
Username: sddm
Username: _mlocate
Username: rtkit
Username: pulse
Username: ldap
Username: tss
Username: openntpd
Username: cups
Username: transmission
Username: tor
Username: mpd


: 1

In [43]:
for item in /tmp/*; do
    echo "$item"
done

/tmp/Temp-488512d8-c936-4b6c-9585-fc7446339a9e
/tmp/Temp-4c203723-8531-4c8d-9642-0ad508721129
/tmp/dropbox-antifreeze-MrxsjD
/tmp/dropbox-antifreeze-beccw3
/tmp/plasma-csd-generator
/tmp/sddm-auth67d4c07b-5760-489a-b242-c56354fe58f4
/tmp/xauth-1000-_0


: 1

In [44]:
for item in /tmp/d*; do
    echo "$item"
done

/tmp/dropbox-antifreeze-MrxsjD
/tmp/dropbox-antifreeze-beccw3


: 1

#### Breaking out of a Loop

In [47]:
for day in Mon Tue Wed Thu Fri; do
    if [ $day == "Thu" ]; then
        break;
    fi
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed


: 1

#### Continuing a Loop

In [49]:
for day in Mon Tue Wed Thu Fri Sat Sun; do
    echo -n "$day"
    if [ $day == "Sat" -o $day == "Sun" ]; then
        echo " (WEEKEND)"
        continue;
    fi
    echo " (weekday)"
done

Mon (weekday)04l
Tue (weekday)
Wed (weekday)
Thu (weekday)
Fri (weekday)
Sat (WEEKEND)
Sun (WEEKEND)


: 1

#### Range of numbers

In [51]:
for num in {1..10}; do
    echo "Number: $num"
done

Number: 1[?2004l
Number: 2
Number: 3
Number: 4
Number: 5


: 1

In [52]:
for num in {1..10..2}; do
    echo "Number: $num"
done

Number: 1[?2004l
Number: 3
Number: 5
Number: 7
Number: 9


: 1

## For Loops Using C-Syntax

In [56]:
for (( i=1; i <= 5; i++ )); do
    echo "Random number $i: $((RANDOM%100))"
done

Random number 1: 502004l
Random number 2: 36
Random number 3: 87
Random number 4: 32
Random number 5: 37


: 1

Interrupt kernel when you're getting bored:

In [59]:
for (( ; ; )); do
    sleep $((RANDOM%10))
    echo "Number: $((i++))"
done

Number: 15?2004l
Number: 16
Number: 17
Number: 18
Number: 19
Number: 20
Number: 21
Number: 22



In [62]:
for ((i=1, j=10; i <= 5 ; i++, j=j+5)); do
    echo "Number $i: $j"
done

Number 1: 10004l
Number 2: 15
Number 3: 20
Number 4: 25
Number 5: 30


: 1

## While Loops

In [63]:
type true

true is a shell builtin


: 1

In [71]:
i=1
while [ $i -le 5 ]; do
  echo "$((i++)) time(s)"
done

1 time(s)[?2004l
2 time(s)
3 time(s)
4 time(s)
5 time(s)


: 1

## Until Loops

In [72]:
i=1
until [ $i -gt 5 ]; do
    echo "$((i++)) time(s)."
done

1 time(s).?2004l
2 time(s).
3 time(s).
4 time(s).
5 time(s).


: 1

## Case Command

General structure:
<code>
case  $variable-name  in
      pattern1|pattern2|pattern3)       
        command1
        ...
        ....
        commandN
        ;;
      pattern4|pattern5|pattern6)
     	command1
        ...
        ....
        commandN
        ;;            
      pattern7|pattern8|patternN)       
     	command1
        ...
        ....
        commandN
        ;;
      *)              
esac
</code>



In [81]:
NOW=$(date +"%a")
case $NOW in
    Mon)
        echo "FULL backup!";;
    Tue|Wed|Thu|Fri)
        echo "PARTIAL backup!";;
    Sat|Sun)
        echo "NO backup!";;
    *) ;;
esac

PARTIAL backup!l


: 1

### Case terminators

The ;& terminator causes case to also execute the next block without testing its pattern. The ;;& operator is like ;; except the case statement doesn't terminate after executing the associated list. Bash just continues testing the next pattern as though the previous pattern didn't match.

## Select Command

Execute this in a real Bash shell:

<code>select state in yes no dk; do
    echo "Your selection is: $state"
    break # without it you have an endless loop
done</code>

# Patterns and Regular Expressions

## Pattern Matching in Bash
1. Globs
2. Extended Globs
3. Brace Expansion
4. Basic Regular Expressions
5. Extended Regular Expressions

### Globs

#### Bash Globbing Options
1. nocaseglob
2. nocasematch
3. nullglob
4. globstar
5. extglob

#### Globs vs. Regular Expressions
Glob<br>
    <code>[0-9]?file*.txt</code><br>
Regular Expression<br>
    <code>[0-9].file.*.txt</code>

1. Globs are faster and less expressive than REs.
2. Globs and REs look similar but are different.
3. Globs match files!
4. REs match text/strings!

Using a Glob with ls<br>
<code>ls [0-9]?file*.txt</code><br>
will be expanded by Bash to:<br>
<code>ls 1_file-rev1.txt 2_file-rev1.txt 3_file-rev1.txt</code><br>
ls is unaware of this expansion!

To achieve the same with RE you had to do something like this:<br>
<code>ls | grep '[0-9].file.*.txt'</code><br>

This command uses REs and globs:<br>
<code>grep '^A.*\.txt' *.txt</code>
Will be expanded by Bash to:<br>
<code>grep '^A.*\.txt'file1.txt file2.txt </code><br>
Then grep will use the RE to search for text in those files. To prevent Bash from interpreting the RE as a glob it has to be enclosed in single quotes!

#### Types of Shell Expansion
Expansions happens in this order:
1. Brace expansion
2. Tilde expansion
3. Parameter expansion
4. Variable expansion 
5. Arithmetic expansion and command substitution from left to right
7. Word splitting
8. Filename expansion (expanding globs into lists)
9. Quote removal

Note: Filename expansion which we think about most is done last!

#### Expansions That Can Change the Number of Words
1. Brace expansion
2. Word splitting
3. Filename expansion

#### Expansions That Do Not Change the Number of Words
1. Arithmetic expansion
2. Command substitution
3. Variable expansion
4. Parameter expansion

#### File Globs

|File Glob|Explanation|
|:--|:--|
|* (Asterisk/Star)|Matches any number of any characters.|
|? (Question mark)|Matches EXACTLY one character - can be any character.|
|[] (Character set)|Matches exactly one character - but we define which ones.|
|[^]|Invert the character set.|
|[!]|Invert the character set. Same as above. Preferred as ^ can also be an anchor meaning beginning of line.|

In [5]:
cd 01

: 1

In [6]:
ls *

 File1.jpg   'file*.txt'   file12.txt   file2.tgz    file6.tgz   file9.txt
 File1.tgz    file-.txt    file14.jpg   file20.jpg   file6.txt   fileC.txt
 File1.txt    file.jpg     file14.tgz   file20.tgz   file7.jpg  'file[.txt'
 File2.jpg    file.tgz     file14.txt   file20.txt   file7.tgz  'file^.txt'
 File2.txt    file.txt     file16.jpg   file3.jpg    file7.txt   filea.txt
 File3.tgz    file10.jpg   file16.tgz   file3.txt    file8.jpg   fileb.txt
 File4.jpg    file10.tgz   file16.txt   file4.tgz    file8.tgz   nocasematch.sh
 File4.txt    file10.txt   file18.jpg   file5.jpg    file8.txt
 File5.tgz    file12.jpg   file18.tgz   file5.txt    file9.jpg
'file .txt'   file12.tgz   file18.txt   file6.jpg    file9.tgz

Upperdir:

alphafiles:
A  B  Ç  Y  Z  á  c  x


: 1

In [7]:
ls *.txt

 File1.txt   'file*.txt'   file12.txt   file20.txt   file7.txt  'file[.txt'
 File2.txt    file-.txt    file14.txt   file3.txt    file8.txt  'file^.txt'
 File4.txt    file.txt     file16.txt   file5.txt    file9.txt   filea.txt
'file .txt'   file10.txt   file18.txt   file6.txt    fileC.txt   fileb.txt


: 1

In [8]:
ls F*

File1.jpg  File1.txt  File2.txt  File4.jpg  File5.tgz
File1.tgz  File2.jpg  File3.tgz  File4.txt


: 1

In [9]:
ls file*.txt

'file .txt'   file10.txt   file18.txt   file6.txt   fileC.txt    fileb.txt
'file*.txt'   file12.txt   file20.txt   file7.txt  'file[.txt'
 file-.txt    file14.txt   file3.txt    file8.txt  'file^.txt'
 file.txt     file16.txt   file5.txt    file9.txt   filea.txt


: 1

In [11]:
ls file?.txt # file.txt is not returned

'file .txt'   file3.txt   file7.txt   fileC.txt    filea.txt
'file*.txt'   file5.txt   file8.txt  'file[.txt'   fileb.txt
 file-.txt    file6.txt   file9.txt  'file^.txt'


: 1

In [13]:
ls file??.txt

file10.txt  file12.txt  file14.txt  file16.txt  file18.txt  file20.txt


: 1

In [15]:
ls file[abc].txt

filea.txt  fileb.txt


: 1

In [16]:
ls file[0-9].txt

file3.txt  file5.txt  file6.txt  file7.txt  file8.txt  file9.txt


: 1

In [17]:
ls file[0-99].txt

file3.txt  file5.txt  file6.txt  file7.txt  file8.txt  file9.txt


: 1

In [18]:
ls file[0-9a].txt

file3.txt  file5.txt  file6.txt  file7.txt  file8.txt  file9.txt  filea.txt


: 1

In [19]:
ls file[a-z9].txt

file9.txt  filea.txt  fileb.txt


: 1

In [20]:
ls file[123456789abc].txt

file3.txt  file6.txt  file8.txt  filea.txt
file5.txt  file7.txt  file9.txt  fileb.txt


: 1

In [21]:
ls file[1-9a-c].txt

file3.txt  file6.txt  file8.txt  filea.txt
file5.txt  file7.txt  file9.txt  fileb.txt


: 1

In [26]:
ls | grep 'file[^a-z]*'

alphafiles
file .txt
file*.txt
file-.txt
file.jpg
file.tgz
file.txt
file10.jpg
file10.tgz
file10.txt
file12.jpg
file12.tgz
file12.txt
file14.jpg
file14.tgz
file14.txt
file16.jpg
file16.tgz
file16.txt
file18.jpg
file18.tgz
file18.txt
file2.tgz
file20.jpg
file20.tgz
file20.txt
file3.jpg
file3.txt
file4.tgz
file5.jpg
file5.txt
file6.jpg
file6.tgz
file6.txt
file7.jpg
file7.tgz
file7.txt
file8.jpg
file8.tgz
file8.txt
file9.jpg
file9.tgz
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt


: 1

In [27]:
ls | grep '^file[^a-z]*' # alphafiles is filtered out.

file .txt
file*.txt
file-.txt
file.jpg
file.tgz
file.txt
file10.jpg
file10.tgz
file10.txt
file12.jpg
file12.tgz
file12.txt
file14.jpg
file14.tgz
file14.txt
file16.jpg
file16.tgz
file16.txt
file18.jpg
file18.tgz
file18.txt
file2.tgz
file20.jpg
file20.tgz
file20.txt
file3.jpg
file3.txt
file4.tgz
file5.jpg
file5.txt
file6.jpg
file6.tgz
file6.txt
file7.jpg
file7.tgz
file7.txt
file8.jpg
file8.tgz
file8.txt
file9.jpg
file9.tgz
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt


: 1

In [29]:
ls | grep '^file[!a-z]*' # Same as previous but maybe not so confusing.

file .txt
file*.txt
file-.txt
file.jpg
file.tgz
file.txt
file10.jpg
file10.tgz
file10.txt
file12.jpg
file12.tgz
file12.txt
file14.jpg
file14.tgz
file14.txt
file16.jpg
file16.tgz
file16.txt
file18.jpg
file18.tgz
file18.txt
file2.tgz
file20.jpg
file20.tgz
file20.txt
file3.jpg
file3.txt
file4.tgz
file5.jpg
file5.txt
file6.jpg
file6.tgz
file6.txt
file7.jpg
file7.tgz
file7.txt
file8.jpg
file8.tgz
file8.txt
file9.jpg
file9.tgz
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt


: 1

In [30]:
ls file[!a-z].txt

'file .txt'   file-.txt   file5.txt   file7.txt   file9.txt  'file[.txt'
'file*.txt'   file3.txt   file6.txt   file8.txt   fileC.txt  'file^.txt'


: 1

#### Special Characters
1. [
2. ]
3. ^
4. !
5. -
If you want to match them you can place them at positions where they cannot be misinterpreted or you can escape them.

In [34]:
ls file[a-z!].txt

bash: !].txt: event not found


: 1

In [36]:
ls file[a-z\!].txt

'file!.txt'   filea.txt   fileb.txt


: 1

In [33]:
ls file[a-z^].txt

'file^.txt'   filea.txt   fileb.txt


: 1

In [37]:
ls file[\!].txt # Match only exclamation mark.

'file!.txt'


: 1

In [38]:
ls file[\^].txt # Match only caret.

'file^.txt'


: 1

In [39]:
ls file[-abc].txt # Matches also a hyphen.

file-.txt  filea.txt  fileb.txt


: 1

In [40]:
ls file[abc-].txt # Matches also a hyphen.

file-.txt  filea.txt  fileb.txt


: 1

In [41]:
ls file[-].txt # Matches a single hyphen.

file-.txt


: 1

In [42]:
ls file-.txt # The same as the previous one.

file-.txt


: 1

In [43]:
ls file[[abc].txt # Matches [. Put it as the first character in the character set.

'file[.txt'   filea.txt   fileb.txt


: 1

In [45]:
ls file[]abc].txt # Brackets lose their special meaning when they are used first in a character set.

filea.txt  fileb.txt


: 1

In [47]:
ls 'file*.txt'

'file*.txt'


: 1

In [49]:
ls file[*abc].txt # Character sets give you more flexibility.

'file*.txt'   filea.txt   fileb.txt


: 1

In [50]:
ls file[ABC].txt

fileC.txt


: 1

In [51]:
ls file[A-Z].txt # The output depends on your collate settings.

fileC.txt


: 1

In [52]:
locale # On my machine it is set to C: LC_COLLATE=C. Which does what you expect.

LANG=en_US.UTF-8
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE=C
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=


: 1

In [53]:
cd Collate; cat file.txt

1[?2004l
2
C
B
2
c
3
A
b


: 1

In [69]:
sort file.txt # With LC_COLLATE=C you get first numbers, then uppercase letters, then lowercase letters.

sort: cannot read: file.txt: No such file or directory


: 1

#### US-UTF8 Collation Sequence
1. If you define the character set [a-z] you define the letters: aAbB...z but without Z.
2. If you define the character set [A-Z] you define the letters: AbBcC..zZ but without z.

In [72]:
LC_COLLATE="en_US.UTF-8" sort file.txt # Uppercase and lowercase letters are intermingled

1[?2004l
2
3
4
5
6
7
8
9
a
A
b
B
c
C
d
D
e
E
f
F
g
G
h
H
i
I
j
J
k
K
l
L
m
M
n
N
o
O
p
P
q
Q
r
R
s
S
t
T
u
U
v
V
w
W
x
X
y
Y
z
Z


: 1

In [74]:
LC_COLLATE=C sort file.txt # Uppercase and lowercase letters are separate.

1[?2004l
2
3
4
5
6
7
8
9
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


: 1

#### Solutions to Inconsistent Sorting
1. Set LC_COLLATE=C in every Bash script.
2. Set globasciiranges shell option (Bash 4.3 and newer)
3. $ bash -O globasciiranges
4. Use POSIX compatible character classes instead of character sets.

#### Using POSIX Compatible Character Classes

In [79]:
cd ../alphafiles

: 1

In [80]:
ls

A  á  B  c  Ç  x  Y  Z


: 1

In [82]:
# Let's use a character set with a character class.
# Upper case characters.
ls [[:upper:]] # Does actually not what was expected. Should print: A  B  Ç   Y  Z


A  B  Y  Z


: 1

In [81]:
# Lower case characters.
ls [[:lower:]] # Does actually not what was expected. Should print: á c x

c  x


: 1

In [83]:
# Numbers
ls [[:digit:]]

ls: cannot access '[[:digit:]]': No such file or directory


: 1

In [84]:
# Upper and lower case
ls [[:alpha:]]

A  B  c  x  Y  Z


: 1

In [85]:
# Upper and lower case plus numbers
ls [[:alnum:]]

A  B  c  x  Y  Z


: 1

In [86]:
# Space, tabs and newlines
ls [[:space:]]

ls: cannot access '[[:space:]]': No such file or directory


: 1

In [87]:
# Printable characters not including spaces
# Is this is why the character classes above did not work? They are not displayable?
ls [[:graph:]]

A  B  c  x  Y  Z


: 1

In [88]:
# Printable characters including spaces
ls [[:print:]]

A  B  c  x  Y  Z


: 1

In [91]:
# Punctuation
ls [[:punct:]]

ls: cannot access '[[:punct:]]': No such file or directory


: 1

In [92]:
# Non-printable control characters
ls [[:cntrl:]]

ls: cannot access '[[:cntrl:]]': No such file or directory


: 1

In [93]:
# Hexadecimal characters
ls [[:xdigit:]]

A  B  c


: 1

In [95]:
ls [[:digit][:space:]]

ls: cannot access '[[:digit][:space:]]': No such file or directory


: 1

In [97]:
# Character class negation
ls [![:digit:][:space:]]

A  B  c  x  Y  Z


: 1

In [98]:
cd ..

: 1

In [99]:
pwd

/home/schmidh/Gitrepos/bash-tutorial/01


: 1

In [101]:
ls file[[:alnum:]].txt

file3.txt  file6.txt  file8.txt  filea.txt  fileC.txt
file5.txt  file7.txt  file9.txt  fileb.txt


: 1

In [102]:
ls file[[:alnum:][:punct:]].txt

 file3.txt   file7.txt   filea.txt  'file!.txt'  'file[.txt'
 file5.txt   file8.txt   fileb.txt  'file*.txt'  'file^.txt'
 file6.txt   file9.txt   fileC.txt   file-.txt


: 1

In [103]:
ls file[![:alnum:][:punct:]].txt

'file .txt'


: 1

In [104]:
ls file[![:alnum:][:space:]].txt

'file!.txt'  'file*.txt'   file-.txt  'file[.txt'  'file^.txt'


: 1

In [105]:
ls file[![:alnum:]].txt

'file .txt'  'file!.txt'  'file*.txt'   file-.txt  'file[.txt'  'file^.txt'


: 1

#### Bash Globbing Options
1. nullglob
2. failglob
3. dotglob
4. globstar
5. nocasematch
6. nocaseglob
7. extglob

To **set** a shell variable use: **shopt -s**<br>
To **unset** a shell variable use: **shopt -u**<br>
To **display** the current value use: **shopt -p**<br>

In [106]:
shopt -p nullglob # nullglob is not set

shopt -u nullglob


: 1

In [111]:
shopt -u nullglob # If there are no *.bin files in the directory
# it will be set to '*.bin' and found to 1
# which in our application case is wrong.
found=0
for i in *.bin; do
    echo $i
    found=1
done
echo $found

*.bin04h
1[?2004h


: 1

In [115]:
shopt -s nullglob # If there are no *.bin files we don't enter the for-loop. This is what we want!
found=0
for i in *.bin; do # pattern is set to null. For-loop is not entered.
    echo $i
    found=1
done
shopt -u nullglob # Immediately deactivate this option! It can break some tools and even shell scripts!
echo $found

0[?2004h


: 1

In [116]:
shopt -p failglob

shopt -u failglob


: 1

In [118]:
shopt -s failglob # If there are no *.bin files an error is displayed.
found=0
for i in *.bin; do # Error is displayed and the code is not executed.
    echo $i
    found=1
done
shopt -u failglob # Immediately deactivate this option! It can break some tools and even shell scripts!
echo $found

bash: no match: *.bin04l
0[?2004h


: 1

In [120]:
shopt -p dotglob

shopt -u dotglob


: 1

In [123]:
shopt -u dotglob

: 1

In [124]:
ls *

 file10.jpg   file16.tgz   file20.txt   file5.jpg   file8.tgz  'file .txt'
 file10.tgz   file16.txt   File2.jpg    File5.tgz   file8.txt  'file!.txt'
 file10.txt   file18.jpg   file2.tgz    file5.txt   file9.jpg  'file*.txt'
 file12.jpg   file18.tgz   File2.txt    file6.jpg   file9.tgz   file-.txt
 file12.tgz   file18.txt   file3.jpg    file6.tgz   file9.txt   file.txt
 file12.txt   File1.jpg    File3.tgz    file6.txt   filea.txt  'file[.txt'
 file14.jpg   File1.tgz    file3.txt    file7.jpg   fileb.txt  'file^.txt'
 file14.tgz   File1.txt    File4.jpg    file7.tgz   fileC.txt   nocasematch.sh
 file14.txt   file20.jpg   file4.tgz    file7.txt   file.jpg
 file16.jpg   file20.tgz   File4.txt    file8.jpg   file.tgz

alphafiles:
A  á  B  c  Ç  x  Y  Z

Collate:
file.txt

Upperdir:


: 1

In [125]:
shopt -s dotglob

: 1

In [126]:
ls * # .hiddenfile is displayed

 file10.jpg   file16.tgz   file20.txt   file5.jpg   file8.tgz  'file .txt'
 file10.tgz   file16.txt   File2.jpg    File5.tgz   file8.txt  'file!.txt'
 file10.txt   file18.jpg   file2.tgz    file5.txt   file9.jpg  'file*.txt'
 file12.jpg   file18.tgz   File2.txt    file6.jpg   file9.tgz   file-.txt
 file12.tgz   file18.txt   file3.jpg    file6.tgz   file9.txt   file.txt
 file12.txt   File1.jpg    File3.tgz    file6.txt   filea.txt  'file[.txt'
 file14.jpg   File1.tgz    file3.txt    file7.jpg   fileb.txt  'file^.txt'
 file14.tgz   File1.txt    File4.jpg    file7.tgz   fileC.txt   .hiddenfile
 file14.txt   file20.jpg   file4.tgz    file7.txt   file.jpg    nocasematch.sh
 file16.jpg   file20.tgz   File4.txt    file8.jpg   file.tgz

alphafiles:
A  á  B  c  Ç  x  Y  Z

Collate:
file.txt

Upperdir:


: 1

In [1]:
shopt -p globstar

shopt -u globstar


: 1

In [2]:
shopt -s globstar

: 1

In [4]:
cd 01

: 1

In [10]:
ls ** # recurses into subdirectories

 Collate/file.txt   alphafiles/Z   file12.jpg   file20.tgz   file8.tgz
 File1.jpg          alphafiles/á   file12.tgz   file20.txt   file8.txt
 File1.tgz          alphafiles/c   file12.txt   file3.jpg    file9.jpg
 File1.txt          alphafiles/x   file14.jpg   file3.txt    file9.tgz
 File2.jpg         'file .txt'     file14.tgz   file4.tgz    file9.txt
 File2.txt         'file!.txt'     file14.txt   file5.jpg    fileC.txt
 File3.tgz         'file*.txt'     file16.jpg   file5.txt   'file[.txt'
 File4.jpg          file-.txt      file16.tgz   file6.jpg   'file^.txt'
 File4.txt          file.jpg       file16.txt   file6.tgz    filea.txt
 File5.tgz          file.tgz       file18.jpg   file6.txt    fileb.txt
 alphafiles/A       file.txt       file18.tgz   file7.jpg    nocasematch.sh
 alphafiles/B       file10.jpg     file18.txt   file7.tgz
 alphafiles/Ç       file10.tgz     file2.tgz    file7.txt
 alphafiles/Y       file10.txt     file20.jpg   file8.jpg

Collate:
file.txt

Upperdir:

alpha

: 1

In [11]:
shopt -s globstar
for i in **; do
    echo $i
done

Collateh
Collate/file.txt
File1.jpg
File1.tgz
File1.txt
File2.jpg
File2.txt
File3.tgz
File4.jpg
File4.txt
File5.tgz
Upperdir
alphafiles
alphafiles/A
alphafiles/B
alphafiles/Ç
alphafiles/Y
alphafiles/Z
alphafiles/á
alphafiles/c
alphafiles/x
file .txt
file!.txt
file .txt file!.txt file*.txt file-.txt file.txt file10.txt file12.txt file14.txt file16.txt file18.txt file20.txt file3.txt file5.txt file6.txt file7.txt file8.txt file9.txt fileC.txt file[.txt file^.txt filea.txt fileb.txt
file-.txt
file.jpg
file.tgz
file.txt
file10.jpg
file10.tgz
file10.txt
file12.jpg
file12.tgz
file12.txt
file14.jpg
file14.tgz
file14.txt
file16.jpg
file16.tgz
file16.txt
file18.jpg
file18.tgz
file18.txt
file2.tgz
file20.jpg
file20.tgz
file20.txt
file3.jpg
file3.txt
file4.tgz
file5.jpg
file5.txt
file6.jpg
file6.tgz
file6.txt
file7.jpg
file7.tgz
file7.txt
file8.jpg
file8.tgz
file8.txt
file9.jpg
file9.tgz
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt
nocasematch.sh


: 1

In [15]:
shopt -s globstar
for i in **/*.txt; do
    echo $i
done

Collate/file.txt
File1.txt
File2.txt
File4.txt
file .txt
file!.txt
file .txt file!.txt file*.txt file-.txt file.txt file10.txt file12.txt file14.txt file16.txt file18.txt file20.txt file3.txt file5.txt file6.txt file7.txt file8.txt file9.txt fileC.txt file[.txt file^.txt filea.txt fileb.txt
file-.txt
file.txt
file10.txt
file12.txt
file14.txt
file16.txt
file18.txt
file20.txt
file3.txt
file5.txt
file6.txt
file7.txt
file8.txt
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt


: 1

In [16]:
cat nocasematch.sh

#!/bin/bash

shopt -s nocasematch 
for file in *; do
	if [[ $file = file*txt ]] ;then
		echo $file
	fi
done




: 1

In [18]:
. ./nocasematch.sh # case insensitive string comparisons because of shopt -s nocasematch

File1.txt
File2.txt
File4.txt
file .txt
file!.txt
file .txt file!.txt file*.txt file-.txt file.txt file10.txt file12.txt file14.txt file16.txt file18.txt file20.txt file3.txt file5.txt file6.txt file7.txt file8.txt file9.txt fileC.txt file[.txt file^.txt filea.txt fileb.txt
file-.txt
file.txt
file10.txt
file12.txt
file14.txt
file16.txt
file18.txt
file20.txt
file3.txt
file5.txt
file6.txt
file7.txt
file8.txt
file9.txt
fileC.txt
file[.txt
file^.txt
filea.txt
fileb.txt


: 1

In [20]:
cd upper*

bash: cd: upper*: No such file or directory


: 1

In [21]:
shopt -s nocaseglob

: 1

In [22]:
cd upper*

: 1

In [2]:
pwd # now it worked

/home/schmidh/Gitrepos/bash-tutorial


: 1

#### Extended Globs

1. Patterns can be more than one character.
2. Match mulitple occurrences.
3. Allow grouping patterns.
4. Nesting pattern groups.
5. Logical AND and OR.

|Pattern|Explanation|Match|Would NOT Match|
|:--|:--|:--|:--|
|photo@(.jpg)|@(match): Match only one occurrence|photo.jpg|photo|
||||photo.jpg.jpg|
|photo@(.jpg\|.png)|Here we add a logical OR|photo.jpg|photo|
|||photo.png|photo.gif|
||||photo.jpg.jpg|
|photo?(.jpg\|.png)|?(match): Match 0 or 1 occurrence|photo|photo.gif|
|||photo.jpg|photo.bmp|
|||photo.png|photo2.jpg|
||||photo.png.png|
||||photo.jpg.jpg|
|photo+.(.jpg\|.png)|+(match): Match 1 or more occurrences|photo.jpg|photo|
|||photo.png|photo.gif|
|||photo.jpg.jpg|photo2.jpg|
|||photo.png.png||
|photo*(.jpg\|.png)|\*(match): Match 0 or more occurrences|photo|photo.gif|
|||photo.jpg|photo2.jpg|
|||photo.png||
|||photo.jpg.jpg||
|||photo.png.png||
|photo!(.jpg\|.png)|!(match): Invert a match|photo|photo.jpg|
|||photo.gif|photo.png|
|||photo2.jpg|document.txt|
|||photo.bmp||
|||photo.txt||
|||photo.jpg.jpg||
|||photo.png.png||
|photo!(?(.jpg\|.png))|!(?(match)): Inverted match 0 or 1 occurrence|photo.gif|photo||
|||photo.bmp|photo.jpg|
|||photo2.jpg|photo.png|
|||photo.png.png||
|||photoe.jpg.jpg||
|!(+(photo\|file)\*+(.jpg\|.gif))|Group matches||||
||Matches all files that do not have photo or file or don't end with jpg or gif.||||

In [4]:
cd 02/globfiles

: 1

In [5]:
ls photo@(.jpg)

photo.jpg


: 1

In [6]:
ls photo.jpg # trivially the same

photo.jpg


: 1

In [8]:
ls photo@(.jpg|.png)

photo.jpg  photo.png


: 1

In [9]:
ls photo?(.jpg|.png)

photo  photo.jpg  photo.png


: 1

In [11]:
ls photo+(.jpg|.png)

photo.jpg  photo.jpg.jpg  photo.png  photo.png.png


: 1

In [12]:
ls photo*(.jpg|.png)

photo  photo.jpg  photo.jpg.jpg  photo.png  photo.png.png


: 1

In [13]:
ls photo!(.jpg|.png)

photo  photo.bmp  photo.gif  photo.jpg.jpg  photo.png.png  photo2.jpg


: 1

In [15]:
ls photo!(?(.jpg|.png)) # The inverse of ls photo(?(.jpg|.png))

photo.bmp  photo.gif  photo.jpg.jpg  photo.png.png  photo2.jpg


: 1

In [16]:
ls !(+(photo|file)*+(.jpg|.gif))

document.txt  photo  photo.bmp  photo.png  photo.png.png


: 1

#### Why You Should Use Extended Globs

1. Make interactive globbing more useful.<br>
Remove all files that start with IMG or DSC but do NOT end in .CRW or .NEF<br>
<code>rm @(IMG|DSC)*!(.CRW|.NEF)</code><br>
This would be difficult to do with the GUI or standard globs.

2. Add more power to if conditional statements.<br>
<code>
if [[ $i = Backup*.@(@(tar\|arc).gz\|tgz) ]]; then
    echo $i
fi</code><br>
Executes much faster than the corresponding RE.

3. Add more power to case statements.<br>
<code>
for file in *; do
    case $file in
      @(IMG\|DSC)\*@(@(.CR2\|.NEF).xmp|.NEF\|.CR2)) (( ALLRAW++ )) ;;&
      DSC*@(.NEF\|.NEF.xmp))                        (( NIKRAW++ )) ;;&
      DSC*@(.NEF.xmp)                               (( NIKSUP++ )) ;;
      IMG*@(.CR2\|.CR2.xmp)                         (( CANRAW++ )) ;;&
      IMG*@(.CR2.xmp)                               (( CANSUP++ )) ;;
      IMG*@(.JPG\|.jpg))                            (( CANJPG++ )) ;;
      *)                                            (( OTHER++  )) ;;
    esac
done
</code>
<br>
We cannot use REs in case statements! This can be epic!

4. Add more power to pattern substitution.<br>
<code>
file=Archive-2017-06-12.tar.gz
echo ${file%%@(.tar\|.bak)*}
-> Archive-2017-06-12</code>

5. Speed!<br>
Extended globs are much faster than REs. So if you can use an extended glob instead of an RE do it!

In [18]:
shopt -p extglob # On my box extended globs are activated by default. If not add them to your ~/.bashrc file.

shopt -s extglob


: 1

In [23]:
pwd

/home/schmidh/Gitrepos/bash-tutorial/02


: 1

In [24]:
cp -Rf goldenfiles/ testfiles

: 1

In [25]:
cd testfiles

: 1

If you can execute the following statements in a real Bash shell.

In [26]:
# ls

Archive-2013-00-1.bak.bz2   Archive-2017-06-13.bak.xz
Archive-2013-00-1.bak.gz    Archive-2017-06-13.tar
Archive-2013-00-1.bak.xz    Archive-2017-06-13.tar.bz2
Archive-2013-00-1.tar       Archive-2017-06-13.tar.gz
Archive-2013-00-1.tar.bz2   Archive-2017-06-13.tar.xz
Archive-2013-00-1.tar.gz    Archive-2017-06-14.bak.bz2
Archive-2013-00-1.tar.xz    Archive-2017-06-14.bak.gz
Archive-2013-00-10.bak.bz2  Archive-2017-06-14.bak.xz
Archive-2013-00-10.bak.gz   Archive-2017-06-14.tar
Archive-2013-00-10.bak.xz   Archive-2017-06-14.tar.bz2
Archive-2013-00-10.tar      Archive-2017-06-14.tar.gz
Archive-2013-00-10.tar.bz2  Archive-2017-06-14.tar.xz
Archive-2013-00-10.tar.gz   Archive-2017-06-15.bak.bz2
Archive-2013-00-10.tar.xz   Archive-2017-06-15.bak.gz
Archive-2013-00-11.bak.bz2  Archive-2017-06-15.bak.xz
Archive-2013-00-11.bak.gz   Archive-2017-06-15.tar
Archive-2013-00-11.bak.xz   Archive-2017-06-15.tar.bz2
Archive-2013-00-11.tar      Archive-2017-06-15.tar.gz
Archive-2013-00-11.tar.bz2  Arch

Archive-2013-00-23.bak.gz   Archive-2017-06-27.tar
Archive-2013-00-23.bak.xz   Archive-2017-06-27.tar.bz2
Archive-2013-00-23.tar      Archive-2017-06-27.tar.gz
Archive-2013-00-23.tar.bz2  Archive-2017-06-27.tar.xz
Archive-2013-00-23.tar.gz   Archive-2017-06-28.bak.bz2
Archive-2013-00-23.tar.xz   Archive-2017-06-28.bak.gz
Archive-2013-00-24.bak.bz2  Archive-2017-06-28.bak.xz
Archive-2013-00-24.bak.gz   Archive-2017-06-28.tar
Archive-2013-00-24.bak.xz   Archive-2017-06-28.tar.bz2
Archive-2013-00-24.tar      Archive-2017-06-28.tar.gz
Archive-2013-00-24.tar.bz2  Archive-2017-06-28.tar.xz
Archive-2013-00-24.tar.gz   Archive-2017-06-29.bak.bz2
Archive-2013-00-24.tar.xz   Archive-2017-06-29.bak.gz
Archive-2013-00-25.bak.bz2  Archive-2017-06-29.bak.xz
Archive-2013-00-25.bak.gz   Archive-2017-06-29.tar
Archive-2013-00-25.bak.xz   Archive-2017-06-29.tar.bz2
Archive-2013-00-25.tar      Archive-2017-06-29.tar.gz
Archive-2013-00-25.tar.bz2  Archive-2017-06-29.tar.xz
Archive-2013-00-25.tar.gz   Arch

Archive-2013-01-1.bak.xz    Archive-2017-07-13.tar.bz2
Archive-2013-01-1.tar       Archive-2017-07-13.tar.gz
Archive-2013-01-1.tar.bz2   Archive-2017-07-13.tar.xz
Archive-2013-01-1.tar.gz    Archive-2017-07-14.bak.bz2
Archive-2013-01-1.tar.xz    Archive-2017-07-14.bak.gz
Archive-2013-01-10.bak.bz2  Archive-2017-07-14.bak.xz
Archive-2013-01-10.bak.gz   Archive-2017-07-14.tar
Archive-2013-01-10.bak.xz   Archive-2017-07-14.tar.bz2
Archive-2013-01-10.tar      Archive-2017-07-14.tar.gz
Archive-2013-01-10.tar.bz2  Archive-2017-07-14.tar.xz
Archive-2013-01-10.tar.gz   Archive-2017-07-15.bak.bz2
Archive-2013-01-10.tar.xz   Archive-2017-07-15.bak.gz
Archive-2013-01-11.bak.bz2  Archive-2017-07-15.bak.xz
Archive-2013-01-11.bak.gz   Archive-2017-07-15.tar
Archive-2013-01-11.bak.xz   Archive-2017-07-15.tar.bz2
Archive-2013-01-11.tar      Archive-2017-07-15.tar.gz
Archive-2013-01-11.tar.bz2  Archive-2017-07-15.tar.xz
Archive-2013-01-11.tar.gz   Archive-2017-07-16.bak.bz2
Archive-2013-01-11.tar.xz   

Archive-2013-01-23.tar      Archive-2017-07-27.tar.gz
Archive-2013-01-23.tar.bz2  Archive-2017-07-27.tar.xz
Archive-2013-01-23.tar.gz   Archive-2017-07-28.bak.bz2
Archive-2013-01-23.tar.xz   Archive-2017-07-28.bak.gz
Archive-2013-01-24.bak.bz2  Archive-2017-07-28.bak.xz
Archive-2013-01-24.bak.gz   Archive-2017-07-28.tar
Archive-2013-01-24.bak.xz   Archive-2017-07-28.tar.bz2
Archive-2013-01-24.tar      Archive-2017-07-28.tar.gz
Archive-2013-01-24.tar.bz2  Archive-2017-07-28.tar.xz
Archive-2013-01-24.tar.gz   Archive-2017-07-29.bak.bz2
Archive-2013-01-24.tar.xz   Archive-2017-07-29.bak.gz
Archive-2013-01-25.bak.bz2  Archive-2017-07-29.bak.xz
Archive-2013-01-25.bak.gz   Archive-2017-07-29.tar
Archive-2013-01-25.bak.xz   Archive-2017-07-29.tar.bz2
Archive-2013-01-25.tar      Archive-2017-07-29.tar.gz
Archive-2013-01-25.tar.bz2  Archive-2017-07-29.tar.xz
Archive-2013-01-25.tar.gz   Archive-2017-07-3.bak.bz2
Archive-2013-01-25.tar.xz   Archive-2017-07-3.bak.gz
Archive-2013-01-26.bak.bz2  Arc

Archive-2013-02-1.tar.bz2   Archive-2017-08-13.tar.xz
Archive-2013-02-1.tar.gz    Archive-2017-08-14.bak.bz2
Archive-2013-02-1.tar.xz    Archive-2017-08-14.bak.gz
Archive-2013-02-10.bak.bz2  Archive-2017-08-14.bak.xz
Archive-2013-02-10.bak.gz   Archive-2017-08-14.tar
Archive-2013-02-10.bak.xz   Archive-2017-08-14.tar.bz2
Archive-2013-02-10.tar      Archive-2017-08-14.tar.gz
Archive-2013-02-10.tar.bz2  Archive-2017-08-14.tar.xz
Archive-2013-02-10.tar.gz   Archive-2017-08-15.bak.bz2
Archive-2013-02-10.tar.xz   Archive-2017-08-15.bak.gz
Archive-2013-02-11.bak.bz2  Archive-2017-08-15.bak.xz
Archive-2013-02-11.bak.gz   Archive-2017-08-15.tar
Archive-2013-02-11.bak.xz   Archive-2017-08-15.tar.bz2
Archive-2013-02-11.tar      Archive-2017-08-15.tar.gz
Archive-2013-02-11.tar.bz2  Archive-2017-08-15.tar.xz
Archive-2013-02-11.tar.gz   Archive-2017-08-16.bak.bz2
Archive-2013-02-11.tar.xz   Archive-2017-08-16.bak.gz
Archive-2013-02-12.bak.bz2  Archive-2017-08-16.bak.xz
Archive-2013-02-12.bak.gz   A

...

Archive-2017-03-8.tar       Backup-2019-11-11.tar.gz
Archive-2017-03-8.tar.bz2   Backup-2019-11-11.tar.xz
Archive-2017-03-8.tar.gz    Backup-2019-11-12.bak.bz2
Archive-2017-03-8.tar.xz    Backup-2019-11-12.bak.gz
Archive-2017-03-9.bak.bz2   Backup-2019-11-12.bak.xz
Archive-2017-03-9.bak.gz    Backup-2019-11-12.tar
Archive-2017-03-9.bak.xz    Backup-2019-11-12.tar.bz2
Archive-2017-03-9.tar       Backup-2019-11-12.tar.gz
Archive-2017-03-9.tar.bz2   Backup-2019-11-12.tar.xz
Archive-2017-03-9.tar.gz    Backup-2019-11-13.bak.bz2
Archive-2017-03-9.tar.xz    Backup-2019-11-13.bak.gz
Archive-2017-04-1.bak.bz2   Backup-2019-11-13.bak.xz
Archive-2017-04-1.bak.gz    Backup-2019-11-13.tar
Archive-2017-04-1.bak.xz    Backup-2019-11-13.tar.bz2
Archive-2017-04-1.tar       Backup-2019-11-13.tar.gz
Archive-2017-04-1.tar.bz2   Backup-2019-11-13.tar.xz
Archive-2017-04-1.tar.gz    Backup-2019-11-14.bak.bz2
Archive-2017-04-1.tar.xz    Backup-2019-11-14.bak.gz
Archive-2017-04-10.bak.bz2  Backup-2019-11-14.b

Archive-2017-04-21.tar.gz   Backup-2019-11-26.bak.bz2
Archive-2017-04-21.tar.xz   Backup-2019-11-26.bak.gz
Archive-2017-04-22.bak.bz2  Backup-2019-11-26.bak.xz
Archive-2017-04-22.bak.gz   Backup-2019-11-26.tar
Archive-2017-04-22.bak.xz   Backup-2019-11-26.tar.bz2
Archive-2017-04-22.tar      Backup-2019-11-26.tar.gz
Archive-2017-04-22.tar.bz2  Backup-2019-11-26.tar.xz
Archive-2017-04-22.tar.gz   Backup-2019-11-27.bak.bz2
Archive-2017-04-22.tar.xz   Backup-2019-11-27.bak.gz
Archive-2017-04-23.bak.bz2  Backup-2019-11-27.bak.xz
Archive-2017-04-23.bak.gz   Backup-2019-11-27.tar
Archive-2017-04-23.bak.xz   Backup-2019-11-27.tar.bz2
Archive-2017-04-23.tar      Backup-2019-11-27.tar.gz
Archive-2017-04-23.tar.bz2  Backup-2019-11-27.tar.xz
Archive-2017-04-23.tar.gz   Backup-2019-11-28.bak.bz2
Archive-2017-04-23.tar.xz   Backup-2019-11-28.bak.gz
Archive-2017-04-24.bak.bz2  Backup-2019-11-28.bak.xz
Archive-2017-04-24.bak.gz   Backup-2019-11-28.tar
Archive-2017-04-24.bak.xz   Backup-2019-11-28.tar.

Archive-2017-04-9.bak.gz    Backup-2019-12-12.tar
Archive-2017-04-9.bak.xz    Backup-2019-12-12.tar.bz2
Archive-2017-04-9.tar       Backup-2019-12-12.tar.gz
Archive-2017-04-9.tar.bz2   Backup-2019-12-12.tar.xz
Archive-2017-04-9.tar.gz    Backup-2019-12-13.bak.bz2
Archive-2017-04-9.tar.xz    Backup-2019-12-13.bak.gz
Archive-2017-05-1.bak.gz    Backup-2019-12-13.bak.xz
Archive-2017-05-1.bak.xz    Backup-2019-12-13.tar
Archive-2017-05-1.tar       Backup-2019-12-13.tar.bz2
Archive-2017-05-1.tar.gz    Backup-2019-12-13.tar.gz
Archive-2017-05-1.tar.xz    Backup-2019-12-13.tar.xz
Archive-2017-05-10.bak.gz   Backup-2019-12-14.bak.bz2
Archive-2017-05-10.bak.xz   Backup-2019-12-14.bak.gz
Archive-2017-05-10.tar      Backup-2019-12-14.bak.xz
Archive-2017-05-10.tar.gz   Backup-2019-12-14.tar
Archive-2017-05-10.tar.xz   Backup-2019-12-14.tar.bz2
Archive-2017-05-11.bak.gz   Backup-2019-12-14.tar.gz
Archive-2017-05-11.bak.xz   Backup-2019-12-14.tar.xz
Archive-2017-05-11.tar      Backup-2019-12-15.bak.

Archive-2017-05-28.bak.xz   Backup-2019-12-26.tar.gz
Archive-2017-05-28.tar      Backup-2019-12-26.tar.xz
Archive-2017-05-28.tar.gz   Backup-2019-12-27.bak.bz2
Archive-2017-05-28.tar.xz   Backup-2019-12-27.bak.gz
Archive-2017-05-29.bak.gz   Backup-2019-12-27.bak.xz
Archive-2017-05-29.bak.xz   Backup-2019-12-27.tar
Archive-2017-05-29.tar      Backup-2019-12-27.tar.bz2
Archive-2017-05-29.tar.gz   Backup-2019-12-27.tar.gz
Archive-2017-05-29.tar.xz   Backup-2019-12-27.tar.xz
Archive-2017-05-3.bak.gz    Backup-2019-12-28.bak.bz2
Archive-2017-05-3.bak.xz    Backup-2019-12-28.bak.gz
Archive-2017-05-3.tar       Backup-2019-12-28.bak.xz
Archive-2017-05-3.tar.gz    Backup-2019-12-28.tar
Archive-2017-05-3.tar.xz    Backup-2019-12-28.tar.bz2
Archive-2017-05-30.bak.gz   Backup-2019-12-28.tar.gz
Archive-2017-05-30.bak.xz   Backup-2019-12-28.tar.xz
Archive-2017-05-30.tar      Backup-2019-12-29.bak.bz2
Archive-2017-05-30.tar.gz   Backup-2019-12-29.bak.gz
Archive-2017-05-30.tar.xz   Backup-2019-12-29.b

: 1

In [30]:
# ls  @(Archive|Backup)*@(.gz|.xz)

: 1

In [28]:
# ls  @(Archive|Backup)-!(2019)*@(.gz|.xz) | sort
# Records from 2019 will still be shown as the asterisk is greedy and matching everything

: 1

In [29]:
# We need something more specific so the asterisk does not match everything.
# ls  @(Archive|Backup)-!(2019)-[0-9][0-9]-*@(.gz|.xz) | sort
# You cannot do this with a standard glob!

: 1

In [39]:
for i in *; do echo ${i#*.}; done | sort -u # Finding the file extensions using substring removal and unique sort

bak.bz2l
bak.gz
bak.xz
tar
tar.bz2
tar.gz
tar.xz
tgz


: 1

In [41]:
# ls -1 @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]-@([0-9]|[0-9][0-9])@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz)

: 1

In [43]:
ls -1 !(*@(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]-@([0-9]|[0-9][0-9])@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz)*)
# These files shouldn't be there.

Archive-2017-05.bak.bz2
Archive-2017-05.tar.bz2


: 1

In [37]:
ls -1 @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz)

Archive-2017-05.bak.bz2
Archive-2017-05.tar.bz2


: 1

In [44]:
for i in @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz); do
    echo $i
done

Archive-2017-05.bak.bz2l
Archive-2017-05.tar.bz2


: 1

In [45]:
for i in @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz); do
    echo ${i%%.*}
done

Archive-2017-05l
Archive-2017-05


: 1

In [47]:
for i in @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz); do
    echo ${i%%.*}-1.
done

Archive-2017-05-1.?2004l
Archive-2017-05-1.


: 1

In [49]:
for i in @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz); do
    echo ${i%%.*}-1.${i#*.}
done

Archive-2017-05-1.bak.bz2
Archive-2017-05-1.tar.bz2


: 1

In [51]:
for i in @(Archive|Backup)-[0-9][0-9][0-9][0-9]-[0-9][0-9]@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz); do
    mv -v $i ${i%%.*}-1.${i#*.}
done

mv: target 'Backup-2019-12-1.bak.xz' is not a directory


: 1

In [52]:
mkdir compressed

: 1

In [53]:
mv @(Archive|Backup)*@(@(.bak|.tar)?(.bz2|.gz|.xz)|.tgz) compressed

: 1

In [54]:
ls -1 compressed | wc -l

2262304l


: 1

In [56]:
cd ..

: 1

In [57]:
rm -Rf testfiles

: 1

In [58]:
cp -Rf goldenfiles/ testfiles

: 1

 ### Extended Globs vs. Regular Expressions
 #### Extended Globs:
 1. match files
 2. for loops
 3. if conditionals
 4. case statements
 5. variable pattern matching
 
 #### Regular Expressions:
 1. match text
 2. grep
 3. awk
 4. sed
 5. if [[ =~ ]]

|Extended Glob|Regular Expression|
|:--|:--|
|@(pattern)|(pattern)|
|?(pattern)|(pattern)?|
(+(pattern)|(pattern)+|
(\*(pattern)|(pattern)\*)|
!(pattern)|(!pattern)|
\*(pattern1\|pattern2)|(pattern1\|pattern2)\*|
|N/A|(pattern){3}|

#### Brace Expansion vs. Globs

1. Globs expand pathnames
2. Brace expansion expands braces
3. Brace expansion is processed first
4. Globs are processed last
5. Brace expansion does NOT depend on file existence
6. Brace expansion is a pure string operation

In [60]:
cd ../03

: 1

In [61]:
touch file{1,2,4}.txt

: 1

In [63]:
ls

file1.txt  file2.txt  file4.txt  permutations


: 1

In [64]:
ls file{1..4}.txt

ls: cannot access 'file3.txt': No such file or directory
file1.txt  file2.txt  file4.txt


: 1

In [65]:
echo file{1..4}.txt

file1.txt file2.txt file3.txt file4.txt


: 1

In [66]:
echo {1..10}

1 2 3 4 5 6 7 8 9 10


: 1

In [67]:
echo {a..c}{10..99}

a10 a11 a12 a13 a14 a15 a16 a17 a18 a19 a20 a21 a22 a23 a24 a25 a26 a27 a28 a29 a30 a31 a32 a33 a34 a35 a36 a37 a38 a39 a40 a41 a42 a43 a44 a45 a46 a47 a48 a49 a50 a51 a52 a53 a54 a55 a56 a57 a58 a59 a60 a61 a62 a63 a64 a65 a66 a67 a68 a69 a70 a71 a72 a73 a74 a75 a76 a77 a78 a79 a80 a81 a82 a83 a84 a85 a86 a87 a88 a89 a90 a91 a92 a93 a94 a95 a96 a97 a98 a99 b10 b11 b12 b13 b14 b15 b16 b17 b18 b19 b20 b21 b22 b23 b24 b25 b26 b27 b28 b29 b30 b31 b32 b33 b34 b35 b36 b37 b38 b39 b40 b41 b42 b43 b44 b45 b46 b47 b48 b49 b50 b51 b52 b53 b54 b55 b56 b57 b58 b59 b60 b61 b62 b63 b64 b65 b66 b67 b68 b69 b70 b71 b72 b73 b74 b75 b76 b77 b78 b79 b80 b81 b82 b83 b84 b85 b86 b87 b88 b89 b90 b91 b92 b93 b94 b95 b96 b97 b98 b99 c10 c11 c12 c13 c14 c15 c16 c17 c18 c19 c20 c21 c22 c23 c24 c25 c26 c27 c28 c29 c30 c31 c32 c33 c34 c35 c36 c37 c38 c39 c40 c41 c42 c43 c44 c45 c46 c47 c48 c49 c50 c51 c52 c53 c54 c55 c56 c57 c58 c59 c60 c61 c62 c63 c64 c65 c66 c67 c68 c69 c70 c71 c72 c73 c74 c75 c76 c77 c78 c79 

: 1

In [68]:
echo {a,b,c}{d,e,f}

ad ae af bd be bf cd ce cf


: 1

In [69]:
echo {1..100..2}

1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47 49 51 53 55 57 59 61 63 65 67 69 71 73 75 77 79 81 83 85 87 89 91 93 95 97 99


: 1

In [70]:
echo {2..100..2}

2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 92 94 96 98 100


: 1

In [71]:
echo {0002..1000..2}

0002 0004 0006 0008 0010 0012 0014 0016 0018 0020 0022 0024 0026 0028 0030 0032 0034 0036 0038 0040 0042 0044 0046 0048 0050 0052 0054 0056 0058 0060 0062 0064 0066 0068 0070 0072 0074 0076 0078 0080 0082 0084 0086 0088 0090 0092 0094 0096 0098 0100 0102 0104 0106 0108 0110 0112 0114 0116 0118 0120 0122 0124 0126 0128 0130 0132 0134 0136 0138 0140 0142 0144 0146 0148 0150 0152 0154 0156 0158 0160 0162 0164 0166 0168 0170 0172 0174 0176 0178 0180 0182 0184 0186 0188 0190 0192 0194 0196 0198 0200 0202 0204 0206 0208 0210 0212 0214 0216 0218 0220 0222 0224 0226 0228 0230 0232 0234 0236 0238 0240 0242 0244 0246 0248 0250 0252 0254 0256 0258 0260 0262 0264 0266 0268 0270 0272 0274 0276 0278 0280 0282 0284 0286 0288 0290 0292 0294 0296 0298 0300 0302 0304 0306 0308 0310 0312 0314 0316 0318 0320 0322 0324 0326 0328 0330 0332 0334 0336 0338 0340 0342 0344 0346 0348 0350 0352 0354 0356 0358 0360 0362 0364 0366 0368 0370 0372 0374 0376 0378 0380 0382 0384 0386 0388 0390 0392 0394 0396 0398 0400 

: 1

In [72]:
echo {a..z}

a b c d e f g h i j k l m n o p q r s t u v w x y z


: 1

In [73]:
echo {a..z..2}

a c e g i k m o q s u w y


: 1

In [74]:
echo {10..0}

10 9 8 7 6 5 4 3 2 1 0


: 1

In [76]:
echo file{,.bak}

file file.bak


: 1

In [79]:
echo /this/is/a/long/path/file{,.bak} # gives you source and destination for cp or mv

/this/is/a/long/path/file /this/is/a/long/path/file.bak


: 1

In [81]:
mkdir bracefiles

: 1

In [82]:
cd bracefiles

: 1

In [85]:
touch Backup-201{1..9}-{0{0..9},1{1,2}}-{1..30}.{tar,bak}.{gz,xz,bz2}

: 1

In [90]:
ls -1 | wc -l

1944004l


: 1

In [91]:
for i in Backup*{0..30..7}.tar.bz2; do
    cp -v $i{,.bak}
done    

'Backup-2011-00-10.tar.bz2' -> 'Backup-2011-00-10.tar.bz2.bak'
'Backup-2011-00-20.tar.bz2' -> 'Backup-2011-00-20.tar.bz2.bak'
'Backup-2011-00-30.tar.bz2' -> 'Backup-2011-00-30.tar.bz2.bak'
'Backup-2011-01-10.tar.bz2' -> 'Backup-2011-01-10.tar.bz2.bak'
'Backup-2011-01-20.tar.bz2' -> 'Backup-2011-01-20.tar.bz2.bak'
'Backup-2011-01-30.tar.bz2' -> 'Backup-2011-01-30.tar.bz2.bak'
'Backup-2011-02-10.tar.bz2' -> 'Backup-2011-02-10.tar.bz2.bak'
'Backup-2011-02-20.tar.bz2' -> 'Backup-2011-02-20.tar.bz2.bak'
'Backup-2011-02-30.tar.bz2' -> 'Backup-2011-02-30.tar.bz2.bak'
'Backup-2011-03-10.tar.bz2' -> 'Backup-2011-03-10.tar.bz2.bak'
'Backup-2011-03-20.tar.bz2' -> 'Backup-2011-03-20.tar.bz2.bak'
'Backup-2011-03-30.tar.bz2' -> 'Backup-2011-03-30.tar.bz2.bak'
'Backup-2011-04-10.tar.bz2' -> 'Backup-2011-04-10.tar.bz2.bak'
'Backup-2011-04-20.tar.bz2' -> 'Backup-2011-04-20.tar.bz2.bak'
'Backup-2011-04-30.tar.bz2' -> 'Backup-2011-04-30.tar.bz2.bak'
'Backup-2011-05-10.tar.bz2' -> 'Backup-2011-05-10.tar.b

'Backup-2014-07-30.tar.bz2' -> 'Backup-2014-07-30.tar.bz2.bak'
'Backup-2014-08-10.tar.bz2' -> 'Backup-2014-08-10.tar.bz2.bak'
'Backup-2014-08-20.tar.bz2' -> 'Backup-2014-08-20.tar.bz2.bak'
'Backup-2014-08-30.tar.bz2' -> 'Backup-2014-08-30.tar.bz2.bak'
'Backup-2014-09-10.tar.bz2' -> 'Backup-2014-09-10.tar.bz2.bak'
'Backup-2014-09-20.tar.bz2' -> 'Backup-2014-09-20.tar.bz2.bak'
'Backup-2014-09-30.tar.bz2' -> 'Backup-2014-09-30.tar.bz2.bak'
'Backup-2014-11-10.tar.bz2' -> 'Backup-2014-11-10.tar.bz2.bak'
'Backup-2014-11-20.tar.bz2' -> 'Backup-2014-11-20.tar.bz2.bak'
'Backup-2014-11-30.tar.bz2' -> 'Backup-2014-11-30.tar.bz2.bak'
'Backup-2014-12-10.tar.bz2' -> 'Backup-2014-12-10.tar.bz2.bak'
'Backup-2014-12-20.tar.bz2' -> 'Backup-2014-12-20.tar.bz2.bak'
'Backup-2014-12-30.tar.bz2' -> 'Backup-2014-12-30.tar.bz2.bak'
'Backup-2015-00-10.tar.bz2' -> 'Backup-2015-00-10.tar.bz2.bak'
'Backup-2015-00-20.tar.bz2' -> 'Backup-2015-00-20.tar.bz2.bak'
'Backup-2015-00-30.tar.bz2' -> 'Backup-2015-00-30.tar.b

'Backup-2018-03-20.tar.bz2' -> 'Backup-2018-03-20.tar.bz2.bak'
'Backup-2018-03-30.tar.bz2' -> 'Backup-2018-03-30.tar.bz2.bak'
'Backup-2018-04-10.tar.bz2' -> 'Backup-2018-04-10.tar.bz2.bak'
'Backup-2018-04-20.tar.bz2' -> 'Backup-2018-04-20.tar.bz2.bak'
'Backup-2018-04-30.tar.bz2' -> 'Backup-2018-04-30.tar.bz2.bak'
'Backup-2018-05-10.tar.bz2' -> 'Backup-2018-05-10.tar.bz2.bak'
'Backup-2018-05-20.tar.bz2' -> 'Backup-2018-05-20.tar.bz2.bak'
'Backup-2018-05-30.tar.bz2' -> 'Backup-2018-05-30.tar.bz2.bak'
'Backup-2018-06-10.tar.bz2' -> 'Backup-2018-06-10.tar.bz2.bak'
'Backup-2018-06-20.tar.bz2' -> 'Backup-2018-06-20.tar.bz2.bak'
'Backup-2018-06-30.tar.bz2' -> 'Backup-2018-06-30.tar.bz2.bak'
'Backup-2018-07-10.tar.bz2' -> 'Backup-2018-07-10.tar.bz2.bak'
'Backup-2018-07-20.tar.bz2' -> 'Backup-2018-07-20.tar.bz2.bak'
'Backup-2018-07-30.tar.bz2' -> 'Backup-2018-07-30.tar.bz2.bak'
'Backup-2018-08-10.tar.bz2' -> 'Backup-2018-08-10.tar.bz2.bak'
'Backup-2018-08-20.tar.bz2' -> 'Backup-2018-08-20.tar.b

'Backup-2012-12-17.tar.bz2' -> 'Backup-2012-12-17.tar.bz2.bak'
'Backup-2012-12-27.tar.bz2' -> 'Backup-2012-12-27.tar.bz2.bak'
'Backup-2012-12-7.tar.bz2' -> 'Backup-2012-12-7.tar.bz2.bak'
'Backup-2013-00-17.tar.bz2' -> 'Backup-2013-00-17.tar.bz2.bak'
'Backup-2013-00-27.tar.bz2' -> 'Backup-2013-00-27.tar.bz2.bak'
'Backup-2013-00-7.tar.bz2' -> 'Backup-2013-00-7.tar.bz2.bak'
'Backup-2013-01-17.tar.bz2' -> 'Backup-2013-01-17.tar.bz2.bak'
'Backup-2013-01-27.tar.bz2' -> 'Backup-2013-01-27.tar.bz2.bak'
'Backup-2013-01-7.tar.bz2' -> 'Backup-2013-01-7.tar.bz2.bak'
'Backup-2013-02-17.tar.bz2' -> 'Backup-2013-02-17.tar.bz2.bak'
'Backup-2013-02-27.tar.bz2' -> 'Backup-2013-02-27.tar.bz2.bak'
'Backup-2013-02-7.tar.bz2' -> 'Backup-2013-02-7.tar.bz2.bak'
'Backup-2013-03-17.tar.bz2' -> 'Backup-2013-03-17.tar.bz2.bak'
'Backup-2013-03-27.tar.bz2' -> 'Backup-2013-03-27.tar.bz2.bak'
'Backup-2013-03-7.tar.bz2' -> 'Backup-2013-03-7.tar.bz2.bak'
'Backup-2013-04-17.tar.bz2' -> 'Backup-2013-04-17.tar.bz2.bak'
'B

'Backup-2016-07-17.tar.bz2' -> 'Backup-2016-07-17.tar.bz2.bak'
'Backup-2016-07-27.tar.bz2' -> 'Backup-2016-07-27.tar.bz2.bak'
'Backup-2016-07-7.tar.bz2' -> 'Backup-2016-07-7.tar.bz2.bak'
'Backup-2016-08-17.tar.bz2' -> 'Backup-2016-08-17.tar.bz2.bak'
'Backup-2016-08-27.tar.bz2' -> 'Backup-2016-08-27.tar.bz2.bak'
'Backup-2016-08-7.tar.bz2' -> 'Backup-2016-08-7.tar.bz2.bak'
'Backup-2016-09-17.tar.bz2' -> 'Backup-2016-09-17.tar.bz2.bak'
'Backup-2016-09-27.tar.bz2' -> 'Backup-2016-09-27.tar.bz2.bak'
'Backup-2016-09-7.tar.bz2' -> 'Backup-2016-09-7.tar.bz2.bak'
'Backup-2016-11-17.tar.bz2' -> 'Backup-2016-11-17.tar.bz2.bak'
'Backup-2016-11-27.tar.bz2' -> 'Backup-2016-11-27.tar.bz2.bak'
'Backup-2016-11-7.tar.bz2' -> 'Backup-2016-11-7.tar.bz2.bak'
'Backup-2016-12-17.tar.bz2' -> 'Backup-2016-12-17.tar.bz2.bak'
'Backup-2016-12-27.tar.bz2' -> 'Backup-2016-12-27.tar.bz2.bak'
'Backup-2016-12-7.tar.bz2' -> 'Backup-2016-12-7.tar.bz2.bak'
'Backup-2017-00-17.tar.bz2' -> 'Backup-2017-00-17.tar.bz2.bak'
'B

'Backup-2011-09-14.tar.bz2' -> 'Backup-2011-09-14.tar.bz2.bak'
'Backup-2011-11-14.tar.bz2' -> 'Backup-2011-11-14.tar.bz2.bak'
'Backup-2011-12-14.tar.bz2' -> 'Backup-2011-12-14.tar.bz2.bak'
'Backup-2012-00-14.tar.bz2' -> 'Backup-2012-00-14.tar.bz2.bak'
'Backup-2012-01-14.tar.bz2' -> 'Backup-2012-01-14.tar.bz2.bak'
'Backup-2012-02-14.tar.bz2' -> 'Backup-2012-02-14.tar.bz2.bak'
'Backup-2012-03-14.tar.bz2' -> 'Backup-2012-03-14.tar.bz2.bak'
'Backup-2012-04-14.tar.bz2' -> 'Backup-2012-04-14.tar.bz2.bak'
'Backup-2012-05-14.tar.bz2' -> 'Backup-2012-05-14.tar.bz2.bak'
'Backup-2012-06-14.tar.bz2' -> 'Backup-2012-06-14.tar.bz2.bak'
'Backup-2012-07-14.tar.bz2' -> 'Backup-2012-07-14.tar.bz2.bak'
'Backup-2012-08-14.tar.bz2' -> 'Backup-2012-08-14.tar.bz2.bak'
'Backup-2012-09-14.tar.bz2' -> 'Backup-2012-09-14.tar.bz2.bak'
'Backup-2012-11-14.tar.bz2' -> 'Backup-2012-11-14.tar.bz2.bak'
'Backup-2012-12-14.tar.bz2' -> 'Backup-2012-12-14.tar.bz2.bak'
'Backup-2013-00-14.tar.bz2' -> 'Backup-2013-00-14.tar.b

'Backup-2013-08-21.tar.bz2' -> 'Backup-2013-08-21.tar.bz2.bak'
'Backup-2013-09-21.tar.bz2' -> 'Backup-2013-09-21.tar.bz2.bak'
'Backup-2013-11-21.tar.bz2' -> 'Backup-2013-11-21.tar.bz2.bak'
'Backup-2013-12-21.tar.bz2' -> 'Backup-2013-12-21.tar.bz2.bak'
'Backup-2014-00-21.tar.bz2' -> 'Backup-2014-00-21.tar.bz2.bak'
'Backup-2014-01-21.tar.bz2' -> 'Backup-2014-01-21.tar.bz2.bak'
'Backup-2014-02-21.tar.bz2' -> 'Backup-2014-02-21.tar.bz2.bak'
'Backup-2014-03-21.tar.bz2' -> 'Backup-2014-03-21.tar.bz2.bak'
'Backup-2014-04-21.tar.bz2' -> 'Backup-2014-04-21.tar.bz2.bak'
'Backup-2014-05-21.tar.bz2' -> 'Backup-2014-05-21.tar.bz2.bak'
'Backup-2014-06-21.tar.bz2' -> 'Backup-2014-06-21.tar.bz2.bak'
'Backup-2014-07-21.tar.bz2' -> 'Backup-2014-07-21.tar.bz2.bak'
'Backup-2014-08-21.tar.bz2' -> 'Backup-2014-08-21.tar.bz2.bak'
'Backup-2014-09-21.tar.bz2' -> 'Backup-2014-09-21.tar.bz2.bak'
'Backup-2014-11-21.tar.bz2' -> 'Backup-2014-11-21.tar.bz2.bak'
'Backup-2014-12-21.tar.bz2' -> 'Backup-2014-12-21.tar.b

'Backup-2015-07-28.tar.bz2' -> 'Backup-2015-07-28.tar.bz2.bak'
'Backup-2015-08-28.tar.bz2' -> 'Backup-2015-08-28.tar.bz2.bak'
'Backup-2015-09-28.tar.bz2' -> 'Backup-2015-09-28.tar.bz2.bak'
'Backup-2015-11-28.tar.bz2' -> 'Backup-2015-11-28.tar.bz2.bak'
'Backup-2015-12-28.tar.bz2' -> 'Backup-2015-12-28.tar.bz2.bak'
'Backup-2016-00-28.tar.bz2' -> 'Backup-2016-00-28.tar.bz2.bak'
'Backup-2016-01-28.tar.bz2' -> 'Backup-2016-01-28.tar.bz2.bak'
'Backup-2016-02-28.tar.bz2' -> 'Backup-2016-02-28.tar.bz2.bak'
'Backup-2016-03-28.tar.bz2' -> 'Backup-2016-03-28.tar.bz2.bak'
'Backup-2016-04-28.tar.bz2' -> 'Backup-2016-04-28.tar.bz2.bak'
'Backup-2016-05-28.tar.bz2' -> 'Backup-2016-05-28.tar.bz2.bak'
'Backup-2016-06-28.tar.bz2' -> 'Backup-2016-06-28.tar.bz2.bak'
'Backup-2016-07-28.tar.bz2' -> 'Backup-2016-07-28.tar.bz2.bak'
'Backup-2016-08-28.tar.bz2' -> 'Backup-2016-08-28.tar.bz2.bak'
'Backup-2016-09-28.tar.bz2' -> 'Backup-2016-09-28.tar.bz2.bak'
'Backup-2016-11-28.tar.bz2' -> 'Backup-2016-11-28.tar.b

: 1

In [92]:
ls *.bak

Backup-2011-00-10.tar.bz2.bak  Backup-2015-06-10.tar.bz2.bak
Backup-2011-00-14.tar.bz2.bak  Backup-2015-06-14.tar.bz2.bak
Backup-2011-00-17.tar.bz2.bak  Backup-2015-06-17.tar.bz2.bak
Backup-2011-00-20.tar.bz2.bak  Backup-2015-06-20.tar.bz2.bak
Backup-2011-00-21.tar.bz2.bak  Backup-2015-06-21.tar.bz2.bak
Backup-2011-00-27.tar.bz2.bak  Backup-2015-06-27.tar.bz2.bak
Backup-2011-00-28.tar.bz2.bak  Backup-2015-06-28.tar.bz2.bak
Backup-2011-00-30.tar.bz2.bak  Backup-2015-06-30.tar.bz2.bak
Backup-2011-00-7.tar.bz2.bak   Backup-2015-06-7.tar.bz2.bak
Backup-2011-01-10.tar.bz2.bak  Backup-2015-07-10.tar.bz2.bak
Backup-2011-01-14.tar.bz2.bak  Backup-2015-07-14.tar.bz2.bak
Backup-2011-01-17.tar.bz2.bak  Backup-2015-07-17.tar.bz2.bak
Backup-2011-01-20.tar.bz2.bak  Backup-2015-07-20.tar.bz2.bak
Backup-2011-01-21.tar.bz2.bak  Backup-2015-07-21.tar.bz2.bak
Backup-2011-01-27.tar.bz2.bak  Backup-2015-07-27.tar.bz2.bak
Backup-2011-01-28.tar.bz2.bak  Backup-2015-07-28.tar.bz2.bak
Backup-2011-01-30.tar.bz2

Backup-2012-03-10.tar.bz2.bak  Backup-2016-09-10.tar.bz2.bak
Backup-2012-03-14.tar.bz2.bak  Backup-2016-09-14.tar.bz2.bak
Backup-2012-03-17.tar.bz2.bak  Backup-2016-09-17.tar.bz2.bak
Backup-2012-03-20.tar.bz2.bak  Backup-2016-09-20.tar.bz2.bak
Backup-2012-03-21.tar.bz2.bak  Backup-2016-09-21.tar.bz2.bak
Backup-2012-03-27.tar.bz2.bak  Backup-2016-09-27.tar.bz2.bak
Backup-2012-03-28.tar.bz2.bak  Backup-2016-09-28.tar.bz2.bak
Backup-2012-03-30.tar.bz2.bak  Backup-2016-09-30.tar.bz2.bak
Backup-2012-03-7.tar.bz2.bak   Backup-2016-09-7.tar.bz2.bak
Backup-2012-04-10.tar.bz2.bak  Backup-2016-11-10.tar.bz2.bak
Backup-2012-04-14.tar.bz2.bak  Backup-2016-11-14.tar.bz2.bak
Backup-2012-04-17.tar.bz2.bak  Backup-2016-11-17.tar.bz2.bak
Backup-2012-04-20.tar.bz2.bak  Backup-2016-11-20.tar.bz2.bak
Backup-2012-04-21.tar.bz2.bak  Backup-2016-11-21.tar.bz2.bak
Backup-2012-04-27.tar.bz2.bak  Backup-2016-11-27.tar.bz2.bak
Backup-2012-04-28.tar.bz2.bak  Backup-2016-11-28.tar.bz2.bak
Backup-2012-04-30.tar.bz2

Backup-2013-06-10.tar.bz2.bak  Backup-2018-00-10.tar.bz2.bak
Backup-2013-06-14.tar.bz2.bak  Backup-2018-00-14.tar.bz2.bak
Backup-2013-06-17.tar.bz2.bak  Backup-2018-00-17.tar.bz2.bak
Backup-2013-06-20.tar.bz2.bak  Backup-2018-00-20.tar.bz2.bak
Backup-2013-06-21.tar.bz2.bak  Backup-2018-00-21.tar.bz2.bak
Backup-2013-06-27.tar.bz2.bak  Backup-2018-00-27.tar.bz2.bak
Backup-2013-06-28.tar.bz2.bak  Backup-2018-00-28.tar.bz2.bak
Backup-2013-06-30.tar.bz2.bak  Backup-2018-00-30.tar.bz2.bak
Backup-2013-06-7.tar.bz2.bak   Backup-2018-00-7.tar.bz2.bak
Backup-2013-07-10.tar.bz2.bak  Backup-2018-01-10.tar.bz2.bak
Backup-2013-07-14.tar.bz2.bak  Backup-2018-01-14.tar.bz2.bak
Backup-2013-07-17.tar.bz2.bak  Backup-2018-01-17.tar.bz2.bak
Backup-2013-07-20.tar.bz2.bak  Backup-2018-01-20.tar.bz2.bak
Backup-2013-07-21.tar.bz2.bak  Backup-2018-01-21.tar.bz2.bak
Backup-2013-07-27.tar.bz2.bak  Backup-2018-01-27.tar.bz2.bak
Backup-2013-07-28.tar.bz2.bak  Backup-2018-01-28.tar.bz2.bak
Backup-2013-07-30.tar.bz2

Backup-2014-09-10.tar.bz2.bak  Backup-2019-03-10.tar.bz2.bak
Backup-2014-09-14.tar.bz2.bak  Backup-2019-03-14.tar.bz2.bak
Backup-2014-09-17.tar.bz2.bak  Backup-2019-03-17.tar.bz2.bak
Backup-2014-09-20.tar.bz2.bak  Backup-2019-03-20.tar.bz2.bak
Backup-2014-09-21.tar.bz2.bak  Backup-2019-03-21.tar.bz2.bak
Backup-2014-09-27.tar.bz2.bak  Backup-2019-03-27.tar.bz2.bak
Backup-2014-09-28.tar.bz2.bak  Backup-2019-03-28.tar.bz2.bak
Backup-2014-09-30.tar.bz2.bak  Backup-2019-03-30.tar.bz2.bak
Backup-2014-09-7.tar.bz2.bak   Backup-2019-03-7.tar.bz2.bak
Backup-2014-11-10.tar.bz2.bak  Backup-2019-04-10.tar.bz2.bak
Backup-2014-11-14.tar.bz2.bak  Backup-2019-04-14.tar.bz2.bak
Backup-2014-11-17.tar.bz2.bak  Backup-2019-04-17.tar.bz2.bak
Backup-2014-11-20.tar.bz2.bak  Backup-2019-04-20.tar.bz2.bak
Backup-2014-11-21.tar.bz2.bak  Backup-2019-04-21.tar.bz2.bak
Backup-2014-11-27.tar.bz2.bak  Backup-2019-04-27.tar.bz2.bak
Backup-2014-11-28.tar.bz2.bak  Backup-2019-04-28.tar.bz2.bak
Backup-2014-11-30.tar.bz2

: 1

In [93]:
mkdir -p 20{10..20}/{01..12}

: 1

In [94]:
tree -d

.
├── 2010
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2011
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2012
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2013
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2014
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2015
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2016
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2017
│ 

: 1

In [95]:
mkdir -p {sales,production,development}/{manager,employee{01..10}}

: 1

In [96]:
tree -d

.
├── 2010
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2011
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2012
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2013
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2014
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2015
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2016
│   ├── 01
│   ├── 02
│   ├── 03
│   ├── 04
│   ├── 05
│   ├── 06
│   ├── 07
│   ├── 08
│   ├── 09
│   ├── 10
│   ├── 11
│   └── 12
├── 2017
│ 

: 1

In [97]:
echo {w,t,}h{e{n{,ce{,forth}},re{,in,fore,with{,al}}},ither,at}

when whence whenceforth where wherein wherefore wherewith wherewithal whither what then thence thenceforth there therein therefore therewith therewithal thither that hen hence henceforth here herein herefore herewith herewithal hither hat


: 1

In [98]:
cd ..

: 1

In [99]:
touch file{1..5}.txt

: 1

### Creating brace expressions AUTOMATICALLY
Press ESC, then CTRL, then open brace '{'<br>
This is the output:<br>
{bracefiles,file{1.txt,2.txt,3.txt,4.txt,5.txt},permutations}

# Parameter Expansion